In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.tree import DecisionTreeRegressor
import os

fileList = os.listdir('/content/drive/MyDrive/prova/dades')
csv_files = [file for file in fileList if file[-3:] == "csv"]
batch_size = 1000

# Cargar y procesar los datos
def load_and_process_data(csv_file):
    data = pd.read_csv(csv_file)
    data.dropna(subset=['num_docks_available', 'Llocs'], inplace=True)
    data = data[data["num_docks_available"] >= 0]
    data = data[data["Llocs"] >= 0]
    data = data[data["num_docks_available"] <= 60]
    data = data[data["Llocs"] <= 60]
    data = data[data["num_docks_available"] <= data["Llocs"]]
    X = data[['station_id', 'hour', 'dayofweek','month', 'Rain_Lectura']]
    Y = (data['num_docks_available'] / data['Llocs']).fillna(0)
    return X, Y

# Cargar y procesar los datos para predicciones
def load_and_process_data2(csv_file):
    data = pd.read_csv(csv_file)
    X = data[['station_id', 'hour', 'dayofweek', 'month', 'Rain_Lectura']]
    return X

# Crear listas para almacenar los lotes de entrenamiento y prueba
X_train_batches = []
Y_train_batches = []

# Iterar sobre los archivos CSV y dividir en lotes de entrenamiento y prueba
for csv_file in csv_files[:]:
    print(csv_file)
    # Cargar y procesar los datos en el lote
    X, Y = load_and_process_data('/content/drive/MyDrive/prova/dades/' + csv_file)

    # Agregar los lotes a las listas
    X_train_batches.append(X)
    Y_train_batches.append(Y)

# Combinar todos los lotes de entrenamiento
X_train = pd.concat(X_train_batches)
Y_train = pd.concat(Y_train_batches)

# Escalar los valores numéricos en el rango [0, 1]
scaler = MinMaxScaler()
X_train_scaled = scaler.fit_transform(X_train)

# Crear un modelo de árbol de decisión
model = DecisionTreeRegressor()

# Entrenar el modelo en los conjuntos de entrenamiento
model.fit(X_train_scaled, Y_train)

# Realizar predicciones con el modelo entrenado
new_data = load_and_process_data2('/content/drive/MyDrive/prova/test/submission.csv')
new_data_scaled = scaler.transform(new_data)
predictions = model.predict(new_data_scaled)

# Obtener los IDs de la columna correspondiente en new_data_scaled
ids = new_data.index.values

# Crear un DataFrame con las predicciones y los IDs
df = pd.DataFrame({'index': ids, 'percentage_docks_available': predictions})

# Definir la ruta y el nombre del archivo CSV
csv_file = 'predicciones2.csv'

# Guardar el DataFrame en el archivo CSV
df.to_csv(csv_file, index=False)
